# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import requests
import gmaps
import os

# Import API key
from api_config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_output_df = pd.read_csv("Cities_Output.csv")
cities_output_df.dropna()
cities_output_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Ponta do Sol,40,PT,1584471858,72,32.67,-17.10,59.00,10.29
1,1,Issia,92,CI,1584472082,52,6.49,-6.59,90.23,2.55
2,2,Bandar-e Lengeh,0,IR,1584472083,68,26.56,54.88,71.60,4.70
3,3,North Battleford,1,CA,1584471926,48,52.78,-108.28,19.99,13.87
4,4,Dekar,68,BW,1584472083,47,-21.53,21.93,74.57,9.51


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

In [4]:
# Store latitude and longitude in locations
locations = cities_output_df[["Lat", "Lng"]]
weight = cities_output_df["Humidity"].astype(float)
#weight = cities_output_df["Humidity"].max()

In [5]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=weight, 
                                 dissipating=False, max_intensity=cities_output_df["Humidity"].max(),
                                 point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
cities_output_df['Hotel Name'] = ""
cities_output_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,0,Ponta do Sol,40,PT,1584471858,72,32.67,-17.10,59.00,10.29,
1,1,Issia,92,CI,1584472082,52,6.49,-6.59,90.23,2.55,
2,2,Bandar-e Lengeh,0,IR,1584472083,68,26.56,54.88,71.60,4.70,
3,3,North Battleford,1,CA,1584471926,48,52.78,-108.28,19.99,13.87,
4,4,Dekar,68,BW,1584472083,47,-21.53,21.93,74.57,9.51,
...,...,...,...,...,...,...,...,...,...,...,...
551,551,Verāval,0,IN,1584472218,47,20.90,70.37,72.23,5.30,
552,552,Tazovsky,100,RU,1584472218,94,67.47,78.70,22.73,17.54,
553,553,Tupaciguara,40,BR,1584472190,37,-18.59,-48.70,86.00,9.17,
554,554,College Station,90,US,1584472219,61,30.63,-96.33,82.99,13.87,


In [7]:
hotel_df = cities_output_df.loc[(cities_output_df['Max Temp'] > 70) &
                                (cities_output_df['Max Temp'] < 77) &
                                (cities_output_df['Wind Speed'] < 10) &
                                (cities_output_df['Cloudiness'] == 0),
                                :].copy()
                           
hotel_df.dropna()
hotel_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
2,2,Bandar-e Lengeh,0,IR,1584472083,68,26.56,54.88,71.60,4.70,
88,88,Lebu,0,CL,1584471647,52,-37.62,-73.65,70.07,9.31,
109,109,Sawākin,0,SD,1584472108,66,19.11,37.33,76.91,8.37,
171,171,Luang Prabang,0,LA,1584472123,69,19.89,102.13,71.33,2.24,
181,181,Gairatganj,0,IN,1584472125,25,23.42,78.24,72.95,9.06,
270,270,Inhambane,0,MZ,1584471955,64,-23.86,35.38,75.20,6.93,
467,467,Naliya,0,IN,1584472196,21,23.27,68.83,70.61,2.53,
486,486,Faqirwali,0,PK,1584472201,32,29.47,73.03,70.43,6.80,
551,551,Verāval,0,IN,1584472218,47,20.90,70.37,72.23,5.30,


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
# find the hotel within 5000 m of coordinates

params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    lat = row['Lat']
    lng = row['Lng']
    
    params['location'] = f"{lat},{lng}"

    response = requests.get(base_url, params=params).json()
    
#    print(json.dumps(response, indent=4, sort_keys=True))
    
    try:
        hotel_df.loc[index, 'Hotel Name'] = response['results'][0]['name']
    except (IndexError):
        print('Missing field/result... skipping.')
    
hotel_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
2,2,Bandar-e Lengeh,0,IR,1584472083,68,26.56,54.88,71.60,4.70,Maryam beauty salon
88,88,Lebu,0,CL,1584471647,52,-37.62,-73.65,70.07,9.31,Hostal Las Lilas
109,109,Sawākin,0,SD,1584472108,66,19.11,37.33,76.91,8.37,Afra Hotel (Alascalh)
171,171,Luang Prabang,0,LA,1584472123,69,19.89,102.13,71.33,2.24,Villa Oasis
181,181,Gairatganj,0,IN,1584472125,25,23.42,78.24,72.95,9.06,new garib namz road lines


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map

# Customize the size of the figure
#figure_layout = {
#    'width': '400px',
#    'height': '300px',
#    'border': '1px solid black',
#    'padding': '1px',
#    'margin': '0 auto 0 auto'
#}

name = hotel_df['Hotel Name'].tolist()
city = hotel_df['City'].tolist()
country = hotel_df['Country'].tolist()

#fig = gmaps.figure(layout=figure_layout)
fig = gmaps.figure()

# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))

In [11]:
# Create a combined map
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))